In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Create Pipeline

In [52]:
from PynPoint2 import Pypeline

pipeline = Pypeline("/Users/markusbonse/Desktop/Workplace_wv",
                    "/Users/markusbonse/Desktop",
                    "/Users/markusbonse/Desktop/Results_ADI_wv")

In [38]:
from PynPoint2 import Pypeline

pipeline = Pypeline("/Users/markusbonse/Desktop/Workplace_no_wv",
                    "/Users/markusbonse/Desktop",
                    "/Users/markusbonse/Desktop/Results_ADI_no_wv")

## Create Data Port

In [39]:
from PynPoint2.core import InputPort

data_port = InputPort("09_star_arr_shifted_01",
                     pipeline.m_data_storage)

In [53]:
from PynPoint2.core import InputPort

data_port = InputPort("10_wavelet_denoised_soft_01",
                     pipeline.m_data_storage)

## Get Planet line

In [54]:
planet_line = data_port[:,66, 48]

## Plot Density

In [61]:
from sklearn.neighbors import KernelDensity

'''
def kde_scipy(x, x_grid, bandwidth=200, **kwargs):
    """Kernel Density Estimation with Scipy"""
    # Note that scipy weights its bandwidth by the covariance of the
    # input data.  To make the results comparable to the other methods,
    # we divide the bandwidth by the sample standard deviation here.
    kde = gaussian_kde(x, bw_method=bandwidth / x.std(ddof=1), **kwargs)
    return kde.evaluate(x_grid)'''


def kde_sklearn(x, x_grid, bandwidth=0.2, **kwargs):
    """Kernel Density Estimation with Scikit-learn"""
    kde_skl = KernelDensity(bandwidth=bandwidth, **kwargs)
    kde_skl.fit(x[:, np.newaxis])
    # score_samples() returns the log-likelihood of the samples
    log_pdf = kde_skl.score_samples(x_grid[:, np.newaxis])
    return np.exp(log_pdf)

In [ ]:
%matplotlib nbagg    
import numpy as np
import matplotlib.pyplot as plt
from sklearn.grid_search import GridSearchCV

# The grid we'll use for plotting
x_grid = np.linspace(-400, 400, 1000)

# estimate bandwidth
from sklearn.grid_search import GridSearchCV
grid = GridSearchCV(KernelDensity(),
                    {'bandwidth': np.linspace(0.2, 20.0, 20)},
                    cv=5)
grid.fit(planet_line[:, None])

print grid.best_params_['bandwidth']

# Plot the kernel density estimate
pdf = kde_sklearn(planet_line,
                  x_grid,
                  bandwidth=grid.best_params_['bandwidth'])
plt.plot(x_grid, pdf, color='blue', alpha=0.5, lw=3)
plt.title("KDE")
plt.xlim(-200, 200)
plt.show()

In [57]:
print x_grid[np.argsort(pdf)[-1]]


print np.mean(planet_line)
print np.median(planet_line)
np.min(planet_line)

-2.4402440244
12.4901597103
6.97906322913


-124.70981423473188

In [71]:
print grid.best_params_['bandwidth']

4.0
